# Rough example usage

In [1]:
%load_ext autoreload
%autoreload 2

from blackboxbandits import compare, bandits, meta, utils
import pandas as pd

# pd.set_option('display.max_rows', None)
# pd.options.display.float_format = '{:20,.2f}'.format

In [5]:
df = compare.BaseOptimizerComparison.get_results_for_dbid(db_root="./saved_results",dbid='bo_20220209_184815_6elli2k5')

In [7]:
df.xs(0,level="study_id")

generalization               \
                                                        achieved     baseline   
optimizer                   function                                            
HyperOpt_0.2.7_a376313      MLP-adam_boston_mse        44.331087    41.404465   
                            MLP-adam_breast_nll         0.221294     0.232888   
                            MLP-adam_diabetes_mse    3250.878187  3291.020021   
                            MLP-adam_digits_nll         0.095667     0.114894   
                            MLP-adam_iris_nll           0.326534     0.231768   
...                                                          ...          ...   
Scikit-GP-LCB_0.9.0_a376313 lasso_breast_nll            0.067619     0.067130   
                            lasso_diabetes_mse       3398.923981  3400.474455   
                            lasso_digits_nll            0.155710     0.155773   
                            lasso_iris_nll              0.106186     0.106329   
                            lasso_wine_nll              0.083685     0.097804   

                                                                          \
                                                           opt     score   
optimizer                   function                                       
HyperOpt_0.2.7_a376313      MLP-adam_boston_mse      31.467371  0.000000   
                            MLP-adam_breast_nll       0.207110  0.449763   
                            MLP-adam_diabetes_mse  3240.537356  0.795161   
                            MLP-adam_digits_nll       0.064641  0.382607   
                            MLP-adam_iris_nll         0.056293  0.000000   
...                                                        ...       ...   
Scikit-GP-LCB_0.9.0_a376313 lasso_breast_nll          0.063883  0.000000   
                            lasso_diabetes_mse     3358.051408  0.036548   
                            lasso_digits_nll          0.154863  0.068377   
                            lasso_iris_nll            0.105172  0.123203   
                            lasso_wine_nll            0.080140  0.799291   

                                                       visible               \
                                                      achieved     baseline   
optimizer                   function                                          
HyperOpt_0.2.7_a376313      MLP-adam_boston_mse      25.855649    33.617917   
                            MLP-adam_breast_nll       0.308906     0.346529   
                            MLP-adam_diabetes_mse  2900.781801  2950.558040   
                            MLP-adam_digits_nll       0.101718     0.119450   
                            MLP-adam_iris_nll         0.382550     0.315071   
...                                                        ...          ...   
Scikit-GP-LCB_0.9.0_a376313 lasso_breast_nll          0.104750     0.103194   
                            lasso_diabetes_mse     2872.355239  3086.369938   
                            lasso_digits_nll          0.161317     0.165168   
                            lasso_iris_nll            0.200487     0.204723   
                            lasso_wine_nll            0.142589     0.146666   

                                                                          
                                                           opt     score  
optimizer                   function                                      
HyperOpt_0.2.7_a376313      MLP-adam_boston_mse      25.628665  0.971589  
                            MLP-adam_breast_nll       0.223942  0.306908  
                            MLP-adam_diabetes_mse  2858.713010  0.541959  
                            MLP-adam_digits_nll       0.092693  0.662728  
                            MLP-adam_iris_nll         0.180899  0.000000  
...                                                        ...       ...  
Scikit-GP-LCB_0.9.0_a376313 lasso_breast_nll          0.099028  0.00

## Run base optimizer comparison

In [2]:
import pmlb
dataset_names = list(map(lambda x: "clf-"+x.replace("_","-"), pmlb.classification_dataset_names)) \
                + list(map(lambda x: "reg-"+x.replace("_","-"), pmlb.regression_dataset_names))

In [3]:
base_comparison = compare.BaseOptimizerComparison(
    optimizers=["HyperOpt", "OpenTuner-BanditA", "OpenTuner-GA",
                "OpenTuner-GA-DE", "PySOT", "RandomSearch",
                "Scikit-GBRT-Hedge", "Scikit-GP-Hedge", "Scikit-GP-LCB"],
    classifiers=["MLP-adam","lasso"],
    datasets = dataset_names,
    metrics=["mse", "nll"],
    num_calls=20,
    num_repetitions=1,
    db_root = "./saved_results",
    datasets_root="./penn_datasets",
    parallel = True
)

In [ ]:
base_comparison.run()
dbid = base_comparison.get_dbid()
dbid

## Run meta optimizer comparison

In [8]:
meta_optimizers = {
    "best_2": meta.BestFixedTAlgos(T=2),
    "best_3": meta.BestFixedTAlgos(T=3),
    "best_4": meta.BestFixedTAlgos(T=4),
    "best_5": meta.BestFixedTAlgos(T=5),
    "best_6": meta.BestFixedTAlgos(T=6),
    "pure_fpml_2": meta.BanditMetaOptimizer(
        bandit_type=bandits.FPML, T=2, S=1
    ),
    "pure_fpml_3": meta.BanditMetaOptimizer(
        bandit_type=bandits.FPML, T=3, S=1
    ),
    "pure_fpml_4": meta.BanditMetaOptimizer(
        bandit_type=bandits.FPML, T=4, S=1
    ),
    "pure_fpml_5": meta.BanditMetaOptimizer(
        bandit_type=bandits.FPML, T=5, S=1
    ),
    "pure_fpml_6": meta.BanditMetaOptimizer(
        bandit_type=bandits.FPML, T=6, S=1
    ),
    "pure_fpml_7": meta.BanditMetaOptimizer(
        bandit_type=bandits.FPML, T=7, S=1
    ),
    "pure_fpmlgr_2": meta.BanditMetaOptimizer(
        bandit_type=bandits.FPMLWithGR, T=2
    ),
    "pure_fpmlgr_3": meta.BanditMetaOptimizer(
        bandit_type=bandits.FPMLWithGR, T=3
    ),
    "pure_fpmlgr_4": meta.BanditMetaOptimizer(
        bandit_type=bandits.FPMLWithGR, T=4
    ),
    "pure_fpmlgr_5": meta.BanditMetaOptimizer(
        bandit_type=bandits.FPMLWithGR, T=5
    ),
    "pure_fpmlgr_6": meta.BanditMetaOptimizer(
        bandit_type=bandits.FPMLWithGR, T=6
    ),
    "streeter_fpml_2x2": meta.BanditMetaOptimizer(
        bandit_type=bandits.StreeterFPML, T=4,
        T_1=2, T_2=2, S=1
    ),
    "streeter_fpml_2x3": meta.BanditMetaOptimizer(
        bandit_type=bandits.StreeterFPML, T=6,
        T_1=2, T_2=3, S=1
    ),
    "streeter_fpml_3x2": meta.BanditMetaOptimizer(
        bandit_type=bandits.StreeterFPML, T=6,
        T_1=3, T_2=2, S=1
    ),
    "streeter_fpmlgr_1x2": meta.BanditMetaOptimizer(
        bandit_type=bandits.StreeterFPML, T=2,
        T_1=1, T_2=2, gr=True
    ),
    "streeter_fpmlgr_1x3": meta.BanditMetaOptimizer(
        bandit_type=bandits.StreeterFPML, T=3,
        T_1=1, T_2=3, gr=True
    ),
    "streeter_fpmlgr_1x4": meta.BanditMetaOptimizer(
        bandit_type=bandits.StreeterFPML, T=4,
        T_1=1, T_2=4, gr=True
    ),
    "streeter_fpmlgr_2x2": meta.BanditMetaOptimizer(
        bandit_type=bandits.StreeterFPML, T=4,
        T_1=2, T_2=2, gr=True
    ),
    "streeter_fpmlgr_1x5": meta.BanditMetaOptimizer(
        bandit_type=bandits.StreeterFPML, T=5,
        T_1=1, T_2=5, gr=True
    ),
    "streeter_fpmlgr_1x6": meta.BanditMetaOptimizer(
        bandit_type=bandits.StreeterFPML, T=6,
        T_1=1, T_2=6, gr=True
    ),
    "streeter_fpmlgr_2x3": meta.BanditMetaOptimizer(
        bandit_type=bandits.StreeterFPML, T=6,
        T_1=2, T_2=3, gr=True
    ),
    "streeter_fpmlgr_3x2": meta.BanditMetaOptimizer(
        bandit_type=bandits.StreeterFPML, T=6,
        T_1=3, T_2=2, gr=True
    ),
    "streeter_exp3_2": meta.BanditMetaOptimizer(
        bandit_type=bandits.Streeter, T=2
    ),
    "streeter_exp3_3": meta.BanditMetaOptimizer(
        bandit_type=bandits.Streeter, T=3
    ),
    "streeter_exp3_4": meta.BanditMetaOptimizer(
        bandit_type=bandits.Streeter, T=4
    ),
    "streeter_exp3_5": meta.BanditMetaOptimizer(
        bandit_type=bandits.Streeter, T=5
    ),
    "streeter_exp3_6": meta.BanditMetaOptimizer(
        bandit_type=bandits.Streeter, T=6
    ),
}

AttributeError: module 'blackboxbandits.bandits' has no attribute 'FPML'

In [32]:
meta_comparison = compare.MetaOptimizerComparison(
    meta_optimizers=meta_optimizers,
    base_optimizers=["HyperOpt", "OpenTuner-BanditA", "OpenTuner-GA",
                     "OpenTuner-GA-DE", "PySOT", "RandomSearch",
                     "Scikit-GBRT-Hedge", "Scikit-GP-Hedge", "Scikit-GP-LCB"],
    classifiers=["MLP-adam","SVM","lasso"],
    datasets = dataset_names,
    metrics=["mse", "nll"],
    num_calls=20,
    num_repetitions=2,
    db_root = "./data",
    datasets_root="./penn_datasets",
    parallel = True,
    num_meta_repetitions = 100
)

In [ ]:
meta_comparison.run_base_comparison()
dbid = meta_comparison.get_dbid()

In [33]:
meta_comparison.load_base_comparison(dbid='bo_20220209_184815_6elli2k5')

In [35]:
meta_comparison.run_meta_comparison()
results = meta_comparison.summary()
results

,visible_score,generalization_score
optimizer,,
HyperOpt_0.2.7_a376313,0.33,0.28
OpenTuner-BanditA_0.8.7_a376313,0.34,0.36
OpenTuner-GA-DE_0.8.7_a376313,0.38,0.28
OpenTuner-GA_0.8.7_a376313,0.25,0.22
PySOT_0.3.3_a376313,0.37,0.30
RandomSearch_0.0.8_a376313,0.19,0.28
Scikit-GBRT-Hedge_0.9.0_a376313,0.49,0.40
Scikit-GP-Hedge_0.9.0_a376313,0.53,0.49
Scikit-GP-LCB_0.9.0_a376313,0.58,0.35


In [55]:
utils.visualise_by_budget(results)

interactive(children=(IntSlider(value=1, description='T', max=7, min=1), Output()), _dom_classes=('widget-inte…